# Imports

In [15]:
import numpy as np
import pickle
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score,KFold
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.callbacks import ModelCheckpoint,CSVLogger
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors, preprocessing, metrics
from sklearn.cross_validation import train_test_split
from keras.models import load_model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Flatten,Dense,Input,Conv2D,MaxPooling2D,GlobalAveragePooling2D,GlobalMaxPool2D
import glob
import pandas as pd
import os
import sys

#  Initialize Random Number Generator

Next, we need to initialize the random number generator to a constant value (7).

This is important to ensure that the results we achieve from this model can be achieved again precisely. It ensures that the stochastic process of training a neural network model can be reproduced.

In [6]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Helper code

In [7]:
def get_proper_images(raw):
    raw_float = np.array(raw, dtype=float) 
    images = raw_float.reshape([-1, 3, 32, 32])
    images = images.transpose([0, 2, 3, 1])
    return images

def onehot_labels(labels):
    return np.eye(100)[labels]

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def print_model(hist):
    # list all data in history
    print(hist.history.keys())
    # summarize history for accuracy
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    

# Load The Dataset

In [8]:
dataset_path = 'datasets/cifar-100-python/'
data = unpickle(dataset_path+'train')
X_train = get_proper_images(unpickle(dataset_path+'train')[b'data'])
Y_train = onehot_labels(unpickle(dataset_path+'train')[b'fine_labels'])
X_test = get_proper_images(unpickle(dataset_path+'test')[b'data'])
Y_test = onehot_labels(unpickle(dataset_path+'test')[b'fine_labels'])

In [9]:
print(data.keys())

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])


In [10]:
print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Defining Hyperparameters

In [19]:
validation_split_param = 0.1
test_size_param = 0.2
num_layers_hidden = 2
activation_output_function = 'softmax'
loss_function_param='categorical_crossentropy'
n_units = 32
epochs_param = 5
batch_size_param = 32
load_weights_flag = 1
learning_rate = 0.1
lr_decay = 1e-6

# Define The Convolutional Neural Networks Model

In [20]:
"""input_shape=(32,32,3)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))
#saved_weights_name='SVMWeights.h5'
#model.load_weights(saved_weights_name)
model.summary()
"""

#def model():
input_shape=(32,32,3)
model = Sequential()
# Adding the input layer
model.add(Conv2D(n_units, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Adding hidden layers
for _ in range(num_layers_hidden-1):
    model.add(Conv2D(n_units*2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())

model.add(Dense(n_units*4, activation='relu'))
# Adding the output layer
model.add(Dense(100, activation=activation_output_function))

#saved_weights_name=dataset_path+'cifar100vgg.h5'
#model.load_weights(saved_weights_name)
print(model.summary())
#return model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               295040    
_________________________________________________________________
dense_6 (Dense)              (None, 100)               12900     
Total para

In [21]:
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
# Compiling Model
model.compile(loss=loss_function_param, optimizer=sgd,metrics=['accuracy'])

checkpoint = ModelCheckpoint(saved_weights_name, 
                                     monitor='val_acc', 
                                     verbose=1, 
                                     save_best_only=True, 
                                     mode='max')
csv_logger = CSVLogger('myLogFile.csv')

In [ ]:
# Fitting Model
model_details = model.fit(X_train,Y_train,
                          epochs=epochs_param,
                          batch_size=batch_size_param,
                         validation_split_param=0.1)

Epoch 1/5
50000/50000 [==============================] - 124s 2ms/step - loss: 15.9571 - acc: 0.0100
Epoch 2/5
50000/50000 [==============================] - 138s 3ms/step - loss: 15.9569 - acc: 0.0100
Epoch 3/5
15776/50000 [========>.....................] - ETA: 1:30 - loss: 15.9608 - acc: 0.0098  

In [ ]:
#saving the model
model.save('model_file.h5')

In [ ]:
#loading the model
my_model = load_model('model_file.h5')

In [ ]:
# making predictions
predictions = my_model.predict(data_to_predict_with)
probability_true = predictions[:,1]

In [ ]:
def model():
    input_shape=(32,32,3)
    model = Sequential()
    # Adding the input layer
    model.add(Conv2D(n_units, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Adding hidden layers
    for _ in range(num_layers_hidden-1):
        model.add(Conv2D(n_units*2, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())

    model.add(Dense(n_units*4, activation='relu'))
    # Adding the output layer
    model.add(Dense(100, activation=activation_output_function))
    #saved_weights_name='SVMWeights.h5'
    #model.load_weights(saved_weights_name)
    print(model.summary())
    return model

In [ ]:
estimator = KerasClassifier(build_fn=model, epochs=epochs_param, batch_size=batch_size_param, verbose=0)

# Evaluate The Model with k-Fold Cross Validation

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))